In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr
from scipy import stats 
import env
import wrangle
import scaling
from itertools import combinations

In [2]:
df = wrangle.wrangle_zillow()

In [3]:
df.shape

(51702, 10)

In [4]:
train, validate, test = wrangle.split_zillow_data(df)

In [5]:
train.shape, validate.shape, test.shape

((28952, 10), (12409, 10), (10341, 10))

In [6]:
baseline = train.taxvaluedollarcnt.mean()
baseline

486872.57325918763

In [7]:
df.bathroomcnt.value_counts()

2.0    21862
3.0    10614
1.0     9533
2.5     3929
4.0     2172
3.5      901
1.5      840
5.0      736
4.5      659
6.0      247
5.5      209
Name: bathroomcnt, dtype: int64

In [8]:
df.head(1)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,parcelid,age,fips2
0,4,2.5,2490,691000,1969,7138.64,Orange,14381521,53,6059


In [9]:
scaler, train_scaled, validate_scaled, test_scaled = scaling.scale_data(train, validate, test, return_scaler=True)


In [10]:
scaler

MinMaxScaler()

In [11]:
train_scaled

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,parcelid,age,fips2
8519,0.8,0.6,0.295896,0.088682,2000,5187.90,LA,11261031,0.117647,6037
20153,0.8,0.4,0.284320,0.403287,1924,19493.37,LA,12033391,0.676471,6037
2353,0.4,0.3,0.151631,0.236105,1966,10221.42,Orange,14294536,0.367647,6059
41153,0.6,0.4,0.188463,0.120497,2014,8989.59,LA,11158611,0.014706,6037
14841,0.6,0.2,0.155458,0.095015,1976,4159.10,Ventura,17273951,0.294118,6111
...,...,...,...,...,...,...,...,...,...,...
15847,0.4,0.2,0.132402,0.026057,1956,1543.72,LA,10817039,0.441176,6037
33650,0.2,0.4,0.164737,0.196863,1942,9147.37,LA,12118110,0.544118,6037
19556,0.4,0.2,0.125706,0.099104,1956,4858.12,Orange,13992985,0.441176,6059
13797,0.4,0.2,0.182818,0.051300,1983,2737.80,LA,10922762,0.242647,6037


In [12]:
from sklearn.preprocessing import PolynomialFeatures


In [17]:
X_train = train_scaled.drop(columns=["taxvaluedollarcnt", 'fips'])
y_train = train_scaled.taxvaluedollarcnt

X_validate = validate_scaled.drop(columns=["taxvaluedollarcnt", 'fips'])
y_validate = validate_scaled.taxvaluedollarcnt

X_test = test_scaled.drop(columns=["taxvaluedollarcnt", 'fips'])
y_test = test_scaled.taxvaluedollarcnt

In [18]:
X_train

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxamount,parcelid,age,fips2
8519,0.8,0.6,0.295896,2000,5187.90,11261031,0.117647,6037
20153,0.8,0.4,0.284320,1924,19493.37,12033391,0.676471,6037
2353,0.4,0.3,0.151631,1966,10221.42,14294536,0.367647,6059
41153,0.6,0.4,0.188463,2014,8989.59,11158611,0.014706,6037
14841,0.6,0.2,0.155458,1976,4159.10,17273951,0.294118,6111
...,...,...,...,...,...,...,...,...
15847,0.4,0.2,0.132402,1956,1543.72,10817039,0.441176,6037
33650,0.2,0.4,0.164737,1942,9147.37,12118110,0.544118,6037
19556,0.4,0.2,0.125706,1956,4858.12,13992985,0.441176,6059
13797,0.4,0.2,0.182818,1983,2737.80,10922762,0.242647,6037


In [19]:
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor

In [20]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train)

# predict train
y_train['predicted_value'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train, y_train.predicted_value)**(1/2)

# predict validate
y_validate['predicted_value'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.predicted_value)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)


NameError: name 'mean_squared_error' is not defined

In [ ]:
df.fips.value_counts()